In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    df = pd.read_excel('b.xlsx')
except FileNotFoundError:
    print("Hata: Excel dosyası bulunamadı.")
    exit()

for index, row in df.iterrows():
    url = row['URLs'] 
    print(80*"-")
    print("Product URL:", url)
   
    
    base_data = {}
    base_data['Product URL'] = url
    base_data['product_title'] = row['Ürün Başlığı']
    base_data['price'] = row['Fiyatı']
    base_data['original_price'] = row['Orijinal Fiyatı']
    base_data['discount_rate'] = row['İndirim Oranı']
    base_data['shipping_cost'] = row['Kargo Ücreti']
    base_data['category'] = row['Kategori']
    base_data['Seo Keyword'] = row['Seo Keyword']
    print("Ürün Başlığı:", base_data['product_title'])
    print("Fiyatı:", base_data['price'])
    print("Orijinal Fiyatı:", base_data['original_price'])
    print("İndirim Oranı:", base_data['discount_rate'])
    print("Kargo Ücreti:", base_data['shipping_cost'])
    print("Kategori:", base_data['category'])
    print("Seo Keyword", base_data['Seo Keyword'])
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Hata:", e)
        continue
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        scripts = soup.find_all('script')

        for script in scripts:
            if 'window.runParams' in script.text:
                run_params_content = script.text
                
                image_urls_match = re.search(r'"imagePathList":(\[.*?\])', run_params_content)
                if image_urls_match:
                    image_urls_str = image_urls_match.group(1)
                    image_urls = json.loads(image_urls_str)
                    image_urls = [url.replace('.png', '.jpg') for url in image_urls[:4]]  # PNG'yi JPG'ye dönüştür
                    base_data['image_urls'] = image_urls 
                    for img_url in image_urls:
                        print("Image URL:", img_url)
               
                if 'productId' in script.text:
                    product_id_match = re.search(r'"productId":"(.*?)"', run_params_content)
                    if product_id_match:
                        base_data['product_id'] = product_id_match.group(1)
                        print("Product ID:", base_data['product_id'])
                        
                pattern = r'\\"productId\\":(\d+)'
                match = re.search(pattern, run_params_content)
                if match:
                    product_id = match.group(1)
                    print("productId:", product_id)
                    base_data['product_id'] = product_id
                else:
                    print("productId bulunamadı.")
                    
                store_name_match = re.search(r'"storeName":"(.*?)"', run_params_content)
                if store_name_match:
                    store_name = store_name_match.group(1)
                    base_data['store_name'] = store_name
                    print("Mağaza Adı:", store_name)
                else:
                    print("storeName bulunamadı.")

                og_title_match = re.search(r'"ogTitle":"(.*?)"', run_params_content)
                if og_title_match:
                    base_data['og_title'] = og_title_match.group(1)
                    title_parts = base_data['og_title'].split('|')
                    if len(title_parts) > 1:
                        print("Title:", title_parts[1].strip())
                    
                keywords_match = re.search(r'"keywords":"(.*?)"', run_params_content)
                if keywords_match:
                    base_data['keywords'] = keywords_match.group(1)
                    print("Keywords:", base_data['keywords'])

                description_match = re.search(r'"description":"(.*?)"', run_params_content)
                if description_match:
                    base_data['description'] = description_match.group(1)
                    print("Description:", base_data['description'])               
                                
                format_trade_count_match = re.search(r'"formatTradeCount":"(.*?)"', run_params_content)
                if format_trade_count_match:
                    base_data['format_trade_count'] = format_trade_count_match.group(1)
                    print("SOLD:", base_data['format_trade_count'])
                
                evarage_star_match = re.search(r'"evarageStar":"(.*?)"', run_params_content)
                if evarage_star_match:
                    base_data['evarage_star'] = evarage_star_match.group(1)
                    print("Average Star:", base_data['evarage_star'])
                
                price_info = re.search(r'"skuAmount":\{"currency":"(.*?)","formatedAmount":"(.*?)","value":(.*?)\}', run_params_content)
                if price_info:
                    base_data['currency'] = price_info.group(1)
                    base_data['formated_price'] = price_info.group(2)
                    base_data['price'] = price_info.group(3)
                    print("Currency:", base_data['currency'])
                    print("Formatted Price:", base_data['formated_price'])
                    print("Price:", base_data['price'])

                discount_info = re.search(r'"skuActivityAmount":\{"currency":"(.*?)","formatedAmount":"(.*?)","value":(.*?)\}', run_params_content)
                if discount_info:
                    base_data['discount_currency'] = discount_info.group(1)
                    base_data['discount_formated_price'] = discount_info.group(2)
                    base_data['discount_price'] = discount_info.group(3)
                    print("Discount Currency:", base_data['discount_currency'])
                    print("Discount Formatted Price:", base_data['discount_formated_price'])
                    print("Discount Price:", base_data['discount_price'])
                
                discount_rate_match = re.search(r'"discount":"(.*?)"', run_params_content)
                if discount_rate_match:
                    base_data['discount_rate'] = discount_rate_match.group(1)
                    print("Discount Rate:", base_data['discount_rate'])
                    
                package_component_match = re.search(r'"packageComponent":\{(.*?)\}', run_params_content)
                if package_component_match:
                    package_component_content = package_component_match.group(1)
                    package_data = {}
                    for item in package_component_content.split(','):
                        key, value = item.split(':')
                        key = key.strip('"')
                        try:
                            value = json.loads(value)
                        except json.JSONDecodeError:
                            value = value.strip('"')
                        package_data[key] = value
                    base_data['package_component'] = package_data
                    for key, value in package_data.items():
                        print(f"{key}: {value}")
                
                if 'productPropComponent' in script.text:
                    prop_component_content = script.text

                    attr_value_matches = re.findall(r'"attrValue":"(.*?)"', prop_component_content)
                    property_value_name_matches = re.findall(r'"attrValue":"(.*?)".*?"attrName":"(.*?)"', prop_component_content)

                    attributes = []
                    for attr_value, attr_name in property_value_name_matches:
                        attribute = {"attrValue": attr_value, "attrName": attr_name}
                        attributes.append(attribute)
                        print(attribute)
                    
                    base_data['attributes'] = attributes
                
                inventory_component_match = re.search(r'"inventoryComponent":\{(.*?)\}', run_params_content)
                if inventory_component_match:
                    inventory_component = inventory_component_match.group(1)

                    sku_tag_match = re.search(r'"skuTag":"(.*?)"', inventory_component)
                    if sku_tag_match:
                        base_data['sku_tag'] = sku_tag_match.group(1)
                        print("SKU Tag:", base_data['sku_tag'])

                    total_quantity_match = re.search(r'"totalQuantity":(\d+)', inventory_component)
                    if total_quantity_match:
                        base_data['total_quantity'] = int(total_quantity_match.group(1))
                        print("total quantity:", base_data['total_quantity'])

                    total_avail_quantity_match = re.search(r'"totalAvailQuantity":(\d+)', inventory_component)
                    if total_avail_quantity_match:
                        base_data['total_avail_quantity'] = int(total_avail_quantity_match.group(1))
                        print("total avail quantity:", base_data['total_avail_quantity'])
                else:
                    print("inventoryComponent bulunamadı.")
                    
                base_data = {
                    "propertyValueId": None,
                    "skuPropertyName": None,
                    "inventory": None,
                    "sku_activity_amount": None,
                    "image_path": None,
                    "detail1_id": None,
                    "detail1": None,
                    "detail2_id": None,
                    "detail2": None
                }

                      
        # Renkler ve fotoğraflar için regex
                color_pattern = re.compile(r'{"skuColorValue":"[^"]+","skuPropertyTips":"[^"]+","propertyValueName":"([^"]+)","propertyValueId":(\d+),"skuPropertyImagePath":"([^"]+)"')
                colors = color_pattern.findall(run_params_content)
                print("Colors:", colors)

                # Bedenler için regex
                size_pattern = re.compile(r'{"skuPropertyTips":"[^"]+","propertyValueName":"([^"]+)","propertyValueId":(\d+)')
                sizes = size_pattern.findall(run_params_content)
                print("Sizes:", sizes)

                # Varyantlar için regex
                variant_pattern = re.compile(r'"skuPropIds":"(\d+),(\d+)".+?"inventory":(\d+).+?"skuActivityAmount":\{"currency":"[^"]+","formatedAmount":"[^"]+","value":([\d\.]+)')
                variants = variant_pattern.findall(run_params_content)
                print("Variants:", variants)

                # İki boyutlu varyantları işleme
                for variant in variants:
                    variant_data = base_data.copy()
                    variant_data['color_id'] = variant[0]
                    variant_data['size_id'] = variant[1]
                    variant_data['inventory'] = variant[2]
                    variant_data['sku_activity_amount'] = variant[3]

                    # Her varyant için ilgili renk ve beden bilgilerini ekle
                    for color in colors:
                        if color[2] == variant[0]:
                            variant_data['color'] = color[0]
                            variant_data['image_path'] = color[2]
                            break

                    for size in sizes:
                        if size[1] == variant[1]:
                            variant_data['size'] = size[0]
                            break

                    # Varyant verilerini JSON olarak yazdır
                    print(json.dumps(variant_data, ensure_ascii=False, indent=4))

                    # Varyant verilerini dosyaya yaz
                    with open('topsellingaliexpress -0-8k.json', 'a', encoding='utf-8') as f:
                        json.dump(variant_data, f, ensure_ascii=False, indent=4)
                        f.write(',\n')

                # Tek boyutlu varyantlarda envanter kontrolü
                single_dimension_inventory_pattern = re.compile(r'"inventory":(\d+).+?"skuActivityAmount":\{"currency":"[^"]+","formatedAmount":"[^"]+","value":([\d\.]+)')

                # Sadece renk bilgisi olan varyantlar
                if colors:
                    for color in colors:
                        variant_data = base_data.copy()
                        variant_data['color_id'] = color[1]
                        variant_data['color'] = color[0]
                        variant_data['image_path'] = color[2]

                        inventory_match = single_dimension_inventory_pattern.search(run_params_content)
                        if inventory_match:
                            variant_data['inventory'] = inventory_match.group(1)
                            variant_data['sku_activity_amount'] = inventory_match.group(2)

                        print(json.dumps(variant_data, ensure_ascii=False, indent=4))

                        with open('topsellingaliexpress -0-8k.json', 'a', encoding='utf-8') as f:
                            json.dump(variant_data, f, ensure_ascii=False, indent=4)
                            f.write(',\n')

                # Sadece beden bilgisi olan varyantlar
                if sizes:
                    for size in sizes:
                        variant_data = base_data.copy()
                        variant_data['size_id'] = size[1]
                        variant_data['size'] = size[0]

                        # Genel bir image_path bulmak için renklerin listesini kontrol et
                        if colors:
                            variant_data['image_path'] = colors[0][2]  # İlk rengin image_path'i

                        inventory_match = single_dimension_inventory_pattern.search(run_params_content)
                        if inventory_match:
                            variant_data['inventory'] = inventory_match.group(1)
                            variant_data['sku_activity_amount'] = inventory_match.group(2)

                        print(json.dumps(variant_data, ensure_ascii=False, indent=4))

                        with open('topsellingaliexpress -0-8k.json', 'a', encoding='utf-8') as f:
                            json.dump(variant_data, f, ensure_ascii=False, indent=4)
                            f.write(',\n')
                else:
                    print("Hata: Sayfa yüklenemedi")
    sleep_time = random.uniform(1, 4)
    print(f"Bekleme süresi: {sleep_time:.2f} saniye")
    time.sleep(sleep_time)


--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005005988698660.html?spm=a2g0o.categorymp.prodcutlist.51.37a1rvgrrvgrk1&pdp_ext_f=%7B%22sku_id%22%3A%2212000035198564762%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Ürün Başlığı: Yaz çiçek şekli saç pençe köpekbalığı firkete tokalarım kadın kızlar saç tokası at kuyruğu tutucu plaj saç aksesuarları
Fiyatı: 13,86TLTL
Orijinal Fiyatı: Orijinal Fiyat Bulunamadı
İndirim Oranı: İndirim Oranı Bulunamadı
Kargo Ücreti: + Kargo: 125,38TL
Kategori: Aksesuarlar
Seo Keyword All Popular,Product,Promotion,Low Price,Great Value,Reviews,China Brands,Blog,Seller Portal,BLACK FRIDAY,AliExpress Asistan
Image URL: https://ae01.alicdn.com/kf/Se2014283781447f6be4a61ce821d4f62U/Yaz-i-ek-ekli-sa-pen-e-k-pekbal-firkete-tokalar-m-kad-n-k-zlar.jpg
Image URL: https://ae01.alicdn.com/kf/Sa07ec09da5c046a9bc4bfd395b4657fdV/Yaz-i-ek-ekli-sa-pen-e-k-pekbal-firkete-tokalar-m-kad-n-k-zlar.jpg

--------------------------------------------------------------------------------
Product URL: https://tr.aliexpress.com/item/1005002240321348.html?spm=a2g0o.categorymp.prodcutlist.482.37a1rvgrrvgrk1&pdp_ext_f=%7B%22sku_id%22%3A%2212000019579331096%22%7D&utparam-url=scene%3Asearch%7Cquery_from%3A
Ürün Başlığı: Okuma gözlüğü mavi ışık engelleme bilgisayar okuyucular kadınlar erkekler için Anti parlama UV filtre gözlük
Fiyatı: 34,29TLTL
Orijinal Fiyatı: 237,61TL
İndirim Oranı: -0.85
Kargo Ücreti: Ücretsiz sevkiyat
Kategori: Aksesuarlar
Seo Keyword All Popular,Product,Promotion,Low Price,Great Value,Reviews,China Brands,Blog,Seller Portal,BLACK FRIDAY,AliExpress Asistan
Image URL: https://ae01.alicdn.com/kf/H8dbfe82eda734d8da041e19c04e8f8046/Okuma-g-zl-mavi-k-engelleme-bilgisayar-okuyucular-kad-nlar-erkekler-i-in-Anti-parlama-UV.jpg
Image URL: https://ae01.alicdn.com/kf/Hcd7f6d30160a4584860c0603c315df8fm/Okuma-g-zl-mavi-k-engelleme-bilgisayar-okuyucular-kad-nlar-erkekler-i-in-Anti-parlama-